In [240]:
import pandas as pd
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

#### Read ad-clicks.csv file

In [241]:
adclicksDF = pd.read_csv('/Users/aloksingh/Downloads/capstone/second/courseraDataSimulation/ad-clicks.csv')
adclicksDF = adclicksDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers
adclicksDF['adCount'] = 1 #each row is a single click, hence add extra column and make it =1

#### Read buy-clicks.csv file

In [242]:
buyClicksDF = pd.read_csv('/Users/aloksingh/Downloads/capstone/second/courseraDataSimulation/buy-clicks.csv')
buyClicksDF = buyClicksDF.rename(columns=lambda x: x.strip()) #remove whitespaces from headers

#### Select user attributes for clustering

In [243]:
userPurchases = buyClicksDF[['userid','price']] #select only userid and price

In [244]:
useradClicks = adclicksDF[['userid','adCount']]

#### Perform aggregation to get total ad-clicks per user

In [245]:
adsPerUser = useradClicks.groupby('userid').sum()
adsPerUser = adsPerUser.reset_index()

#### Perform aggregation to get total revenue generated per user

In [246]:
revenuePerUser = userPurchases.groupby('userid').sum()
revenuePerUser = revenuePerUser.reset_index()

In [247]:
trainingDF = adsPerUser.merge(revenuePerUser, on='userid') #userid, adCount, price

#### Remove userid before training and keep other two attributes

In [248]:
pDF = sqlContext.createDataFrame(trainingDF)

In [249]:
parsedData = pDF.rdd.map(lambda line: array([line[1], line[2]])) #adCount, price (i.e. revenue)

#### Train KMeans model to create two clusters

In [250]:
clusters = KMeans.train(parsedData, 2, maxIterations=10, runs=10, initializationMode="random")

#### Display the centers of two clusters

In [251]:
clusters.centers

[array([ 41.29090909,  61.92224242]), array([ 25.15489749,  15.48749431])]

#### Analyze the cluster centers: 
#### Cluster 0 is centered at [ 41.29090909, 61.92224242 ] 
#### Cluster 1 is centered at [ 25.15489749, 15.48749431 ] 
#### Cluster 0 players click on ads more often and spend more money on in-app purchases